In [8]:
import os
import math
import random
import csv
import pickle
import shutil

import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
from gurobipy import Model, GRB, quicksum, GurobiError
import numpy as np
from scipy.sparse import lil_matrix, save_npz

from IPython.display import HTML
import matplotlib.animation as animation
from matplotlib.collections import LineCollection

# ------------------------------
# Helper Functions
# ------------------------------

def create_random_subgraph(G, radius=500, weight='length'):
    """Pick a random center node and return a subgraph of nodes reachable within radius."""
    nodes = list(G.nodes())
    center_node = random.choice(nodes)
    subgraph = ox.truncate.truncate_graph_dist(G, center_node, dist=radius, weight=weight)
    return subgraph

def save_subgraph(graph, filename):
    """Save the graph in GraphML format (reloadable by OSMnx)."""
    ox.save_graphml(graph, filename)

def get_edge_geometry(graph, u, v):
    """
    Given a graph and an edge from u to v, return a list of (x, y) points.
    If the edge has a 'geometry' attribute, use it; otherwise return a straight line.
    """
    edge_data = graph.get_edge_data(u, v)
    if edge_data is None:
        return [(graph.nodes[u]['x'], graph.nodes[u]['y']),
                (graph.nodes[v]['x'], graph.nodes[v]['y'])]
    first_key = list(edge_data.keys())[0]
    data = edge_data[first_key]
    if 'geometry' in data:
        coords = list(data['geometry'].coords)
    else:
        coords = [(graph.nodes[u]['x'], graph.nodes[u]['y']),
                  (graph.nodes[v]['x'], graph.nodes[v]['y'])]
    return coords

def plot_single_truck_route_from_model(graph, truck_id, route, depot):
    """
    Visualize a single truck's route on the given graph using the explicit route output.
    
    Parameters:
      graph    : a networkx graph (e.g., from OSMnx)
      truck_id : truck id
      route    : route list for the truck, of the form [depot, (u,v), (u,v), ..., depot]
      depot    : depot node (int) for this truck
    
    The function highlights edges that appear in the route list in red (with thicker linewidth)
    and leaves other edges gray.
    
    Returns (fig, ax).
    """
    G_copy = graph.copy()
    service_edges = set()
    for item in route:
        if isinstance(item, tuple):
            service_edges.add(item)
    for (u, v, key, data) in G_copy.edges(keys=True, data=True):
        if (u, v) in service_edges:
            data["color"] = "red"
            data["linewidth"] = 3.0
        else:
            data["color"] = "gray"
            data["linewidth"] = 1.0
    edge_colors = [data.get("color", "gray") for (_,_,_,data) in G_copy.edges(keys=True, data=True)]
    edge_linewidths = [data.get("linewidth", 1.0) for (_,_,_,data) in G_copy.edges(keys=True, data=True)]
    fig, ax = ox.plot_graph(G_copy, node_size=20,
                            edge_color=edge_colors, edge_linewidth=edge_linewidths,
                            show=False, close=False)
    sx = G_copy.nodes[depot]['x']
    sy = G_copy.nodes[depot]['y']
    ax.scatter(sx, sy, marker='*', c='yellow', s=300, edgecolors='black', zorder=5)
    ax.set_title(f"Truck {truck_id} Route")
    return fig, ax

def plot_truck_routes_from_model(graph, route_dict, depots):
    """
    Visualize truck routes on the given graph using the explicit route output.
    
    Parameters:
      graph      : a networkx graph (e.g., from OSMnx)
      route_dict : dictionary mapping truck id to a route list.
                   Each route list is of the form:
                     [depot, (u,v), (u,v), ..., depot]
                   where the first and last values are depot nodes (ints),
                   and the intermediate tuples represent edges to be highlighted.
      depots     : dictionary mapping truck id to depot node.
    
    The function colors each edge in the graph: if the edge (u,v) appears in the route list,
    it is colored red with a thicker linewidth; otherwise, it is colored gray.
    Each truck’s route is shown in its own subplot with a unique color.
    
    Returns (fig, ax).
    """
    trucks = list(route_dict.keys())
    if not trucks:
        print("No routes found; nothing to plot.")
        return
    n = len(trucks)
    cols = int(math.ceil(math.sqrt(n)))
    rows = int(math.ceil(n / cols))
    fig, axes = plt.subplots(rows, cols, figsize=(7 * cols, 7 * rows))
    if n == 1:
        axes = [[axes]]
    elif rows == 1:
        axes = [axes]
    axes_flat = [ax for row in axes for ax in row]
    colors = ['red', 'blue', 'green', 'orange', 'purple']
    
    for idx, truck in enumerate(trucks):
        ax = axes_flat[idx]
        G_copy = graph.copy()
        service_edges = set()
        route = route_dict[truck]
        for item in route:
            if isinstance(item, tuple):
                service_edges.add(item)
        # Color edges based on whether they appear in service_edges.
        for (u, v, key, data) in G_copy.edges(keys=True, data=True):
            if (u, v) in service_edges:
                data["color"] = colors[truck % len(colors)]
                data["linewidth"] = 3.0
            else:
                data["color"] = "gray"
                data["linewidth"] = 1.0
        edge_colors = [data.get("color", "gray") for (_, _, _, data) in G_copy.edges(keys=True, data=True)]
        edge_linewidths = [data.get("linewidth", 1.0) for (_, _, _, data) in G_copy.edges(keys=True, data=True)]
        ox.plot_graph(G_copy, ax=ax, node_size=20,
                      edge_color=edge_colors, edge_linewidth=edge_linewidths,
                      show=False, close=False)
        ax.set_title(f"Truck {truck}")
        depot_node = depots[truck]
        sx = G_copy.nodes[depot_node]['x']
        sy = G_copy.nodes[depot_node]['y']
        ax.scatter(sx, sy, marker='*', c='yellow', s=300, edgecolors='black', zorder=5)
        ax.set_facecolor('black')
    for idx in range(n, rows * cols):
        axes_flat[idx].axis("off")
    plt.tight_layout()
    plt.show()
    return fig, ax

def animate_truck_routes(graph, route_dict, depots, interval=1000):
    """
    Animate truck routes on the given graph.
    
    Parameters:
      graph      : a networkx graph with 'x' and 'y' attributes.
      route_dict : dictionary mapping truck id to a route list.
                   Each route list is [depot, (u,v), (u,v), ..., depot].
      depots     : dictionary mapping truck id to depot node.
      interval   : frame delay in ms.
    
    This function draws the full graph, marks the depot as a yellow star,
    highlights traversed edges using their actual geometry (in red),
    and for the current edge marks the starting node (yellow circle) and the ending node (green pentagon).
    
    Returns the animation object.
    """
    trucks = list(route_dict.keys())
    if not trucks:
        print("No routes found; nothing to animate.")
        return None
    n = len(trucks)
    cols = int(math.ceil(math.sqrt(n)))
    rows = int(math.ceil(n / cols))
    fig, axes = plt.subplots(rows, cols, figsize=(7 * cols, 7 * rows))
    if n == 1:
        axes = [axes]
    elif rows == 1:
        axes = axes
    else:
        axes = [ax for row in axes for ax in row]
    
    truck_routes = {}   # store route for each truck
    dyn_markers = {}    # dynamic markers for current edge
    persistent_lines = {}  # persistent LineCollection for traversed edges
    route_color = 'red'  # persistent route highlighted in red
    
    for idx, truck in enumerate(trucks):
        ax = axes[idx]
        ox.plot_graph(graph, ax=ax, node_size=20, edge_linewidth=1.0, show=False, close=False)
        ax.set_title(f"Truck {truck}")
        ax.set_facecolor('black')
        depot_node = depots[truck]
        depot_x = graph.nodes[depot_node]['x']
        depot_y = graph.nodes[depot_node]['y']
        ax.scatter(depot_x, depot_y, marker='*', c='yellow', s=300, edgecolors='black', zorder=5)
        truck_routes[truck] = route_dict[truck]
        start_marker, = ax.plot([], [], 'o', color='yellow', markersize=10, zorder=11)
        end_marker, = ax.plot([], [], 'p', color='green', markersize=10, zorder=11)
        start_marker.set_animated(True)
        end_marker.set_animated(True)
        dyn_markers[truck] = (start_marker, end_marker)
        persistent_line = LineCollection([], colors=[route_color], linewidths=3, zorder=9)
        persistent_lines[truck] = persistent_line
        ax.add_collection(persistent_line)
    
    max_frames = max(len(route) for route in truck_routes.values())
    
    def update(frame):
        artists = []
        for truck in trucks:
            route = truck_routes[truck]
            segments = []
            for i in range(1, min(frame+1, len(route))):
                elem = route[i]
                if isinstance(elem, tuple):
                    u, v = elem
                    seg = get_edge_geometry(graph, u, v)
                    segments.append(seg)
            persistent_lines[truck].set_segments(segments)
            start_marker, end_marker = dyn_markers[truck]
            if frame < len(route) - 1:
                current_elem = route[frame]
                next_elem = route[frame + 1]
                if isinstance(current_elem, tuple):
                    s_node = current_elem[0]
                else:
                    s_node = current_elem
                if isinstance(next_elem, tuple):
                    e_node = next_elem[1]
                else:
                    e_node = next_elem
                s_x = graph.nodes[s_node]['x']
                s_y = graph.nodes[s_node]['y']
                e_x = graph.nodes[e_node]['x']
                e_y = graph.nodes[e_node]['y']
                start_marker.set_data([s_x], [s_y])
                end_marker.set_data([e_x], [e_y])
            else:
                start_marker.set_data([], [])
                end_marker.set_data([], [])
            artists.extend([persistent_lines[truck], start_marker, end_marker])
        return artists
    
    ani = animation.FuncAnimation(fig, update, frames=max_frames, interval=interval, blit=True)
    plt.tight_layout()
    return ani

# ------------------------------
# Integrated Ground Truth Generator
# ------------------------------

def ground_truth_generator_from_gurobi(place_name, radius, min_trucks, max_trucks, num_samples, variations_per_sample, seed=10):
    """
    Generates ground truth samples using the provided Gurobi arc routing model.
    For each sample, the function:
      - Loads a map via OSMnx using place_name.
      - Creates a random subgraph (using the given radius) and saves it in GraphML format.
      - Precomputes deadhead distances using the provided formulation.
      - Builds and solves the Gurobi model (with decision variables x, ordering variables u, and explicit z).
      - Uses the active arcs (x[i,j,k]=1) to build a route_components dictionary.
      - Constructs connected truck_paths from route_components.
      - Computes, for each truck:
            * Deadhead distance (sum over consecutive arcs via d_arc)
            * Plowing distance (sum over plow_cost for each task edge)
            * Total distance (deadhead + plowing)
            * Deadhead edges (list of arcs used as deadhead)
      - Saves a CSV file (aggregated in the root folder) with columns:
            Number of nodes,
            Number of edges,
            List of nodes,
            List of edges,
            List of edge lengths,
            Number of trucks,
            List of truck starts (depot nodes),
            List of task lists (edges to be plowed),
            List of task nodes (nodes of the task list),
            Truck paths,
            List of total distances travelled by each truck,
            The longest individual distance travelled,
            The longest individual deadhead distance travelled (z),
            List of deadhead distances of each truck,
            List of edges on which the deadhead lie,
            Optimal objective value.
      - Saves model files (LP and MPS) and the coefficient matrix (pickle and NPZ).
      - In case the model is too large or infeasible, saves an image of the subgraph (with depot markers) with an appropriate filename.
      - Generates an animation (using animate_truck_routes) for each sample subgraph and saves it as an MP4 file.
    
    All outputs are stored in a folder "ground_truth_from_gurobi".
    """
    random.seed(seed)
    print("Loading map for", place_name)
    full_graph = ox.graph_from_place(place_name, network_type="drive")
    print("Map loaded.")
    
    root_folder = "final_ground_truth_from_gurobi"
    if not os.path.exists(root_folder):
        os.makedirs(root_folder)
        
    os.makedirs(root_folder, exist_ok=True)

    
    csv_filename = os.path.join(root_folder, "final_ground_truth_samples.csv")
    first_write = not os.path.exists(csv_filename)
    csvfile = open(csv_filename, "a", newline="")
    writer = csv.writer(csvfile)
    
    if first_write:
        header = [
            "Sample ID", "Number of Nodes", "Number of Edges", "List of Nodes", "List of Edges",
            "List of Edge Lengths", "Number of Trucks", "Truck Starts", "Task List", "Task Nodes",
            "Truck Paths", "Truck Total Distances", "Longest Total Distance",
            "Longest Deadhead (z)", "Truck Deadhead Distances", "Truck Deadhead Edges",
            "Optimal Objective Value"
        ]
        writer.writerow(header)
        csvfile.flush()
    
    valid_sample_count = 0
    
    for sample_idx in range(num_samples):
        sample_id = f"subgraph_{sample_idx}"
        print(f"\nGenerating sample {sample_id}...")
        subgraph = create_random_subgraph(full_graph, radius=radius, weight='length')
        nodes_sub = list(subgraph.nodes())
        edges_sub = list(subgraph.edges())
        num_nodes = len(nodes_sub)
        num_edges = len(edges_sub)
        print(f"Sample {sample_id}: {num_nodes} nodes, {num_edges} edges.")
        
        # Save subgraph as GraphML.
        graphml_filename = os.path.join(root_folder, f"{sample_id}.graphml")
        ox.save_graphml(subgraph, graphml_filename)
        
        if num_edges == 0:
            print(f"Sample {sample_id}: No edges found. Skipping sample.")
            continue
        
        local_min_trucks = max(min_trucks, 2)
        if local_min_trucks >= num_nodes/3 or max_trucks >= num_nodes/3:
            new_max = max(2, int(num_nodes/3)) - 1
            print(f"Sample {sample_id}: Truck range exceeds 1/3 of nodes. Adjusting min_trucks to 2 and max_trucks to {new_max}.")
            local_min_trucks = 2
            local_max_trucks = new_max
        else:
            local_max_trucks = max_trucks
        if local_max_trucks < local_min_trucks:
            local_max_trucks = local_min_trucks
        
        sample_folder = os.path.join(root_folder, sample_id)
        if not os.path.exists(sample_folder):
            os.makedirs(sample_folder)
        
        # Save plain subgraph image.
        fig_plain, ax_plain = ox.plot_graph(subgraph, figsize=(6,6), node_size=5, edge_linewidth=0.5, show=False, close=False)
        plain_img_filename = os.path.join(sample_folder, f"{sample_id}_subgraph.png")
        fig_plain.savefig(plain_img_filename)
        plt.close(fig_plain)
        
        valid_variation_found = False
        
        for var_idx in range(variations_per_sample):
            variation_id = f"{sample_id}_variation_{var_idx}"
            print(f"Generating variation {variation_id}...")
            num_trucks_chosen = random.randint(local_min_trucks, local_max_trucks)
            print(f"Using {num_trucks_chosen} trucks.")
            depot_nodes = random.sample(nodes_sub, num_trucks_chosen)
            tasks = edges_sub.copy()
            edge_lengths = []
            for (u, v) in tasks:
                try:
                    data = next(iter(subgraph[u][v].values()))
                    L = data.get("length", 1.0)
                except Exception:
                    L = 1.0
                edge_lengths.append(L)
            task_nodes = list(set([u for (u,v) in tasks] + [v for (u,v) in tasks]))
            plow_cost = {t: edge_lengths[idx] for idx, t in enumerate(tasks)}
            
            nodes_dict = {}
            for k in range(num_trucks_chosen):
                nodes_dict[k] = tasks.copy()
                nodes_dict[k].append(depot_nodes[k])
            
            print(f"Variation {variation_id}: Building model...")
            m = Model(variation_id)
            M_val = 1e6
            x = {}
            for k in range(num_trucks_chosen):
                for i in nodes_dict[k]:
                    for j in nodes_dict[k]:
                        if i == j:
                            continue
                        x[i, j, k] = m.addVar(vtype=GRB.BINARY, name=f"x_{i}_{j}_{k}")
            u_vars = {}
            for t in tasks:
                u_vars[t] = m.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"u_{t}")
            z = m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="z")
            m.update()
            
            d_task = {}
            for t1 in tasks:
                for t2 in tasks:
                    if t1 == t2:
                        continue
                    try:
                        d_task[(t1, t2)] = nx.shortest_path_length(subgraph, source=t1[1], target=t2[0], weight="length")
                    except nx.NetworkXNoPath:
                        d_task[(t1, t2)] = 1e6
            d_dep_start = {}
            d_end_dep = {}
            for k in range(num_trucks_chosen):
                dpt = depot_nodes[k]
                for t in tasks:
                    try:
                        d_dep_start[(k, t)] = nx.shortest_path_length(subgraph, source=dpt, target=t[0], weight="length")
                    except nx.NetworkXNoPath:
                        d_dep_start[(k, t)] = 1e6
                    try:
                        d_end_dep[(k, t)] = nx.shortest_path_length(subgraph, source=t[1], target=dpt, weight="length")
                    except nx.NetworkXNoPath:
                        d_end_dep[(k, t)] = 1e6
            d_arc = {}
            for k in range(num_trucks_chosen):
                for i in nodes_dict[k]:
                    for j in nodes_dict[k]:
                        if i == j:
                            continue
                        if isinstance(i, int):
                            d_arc[(i, j, k)] = d_dep_start[(k, j)]
                        elif isinstance(j, int):
                            d_arc[(i, j, k)] = d_end_dep[(k, i)]
                        else:
                            d_arc[(i, j, k)] = d_task[(i, j)]
            
            obj_expr = quicksum(d_arc[(i, j, k)] * x[i, j, k]
                                for k in range(num_trucks_chosen)
                                for i in nodes_dict[k]
                                for j in nodes_dict[k] if i != j)
            c_LOS = 1
            m.setObjective(obj_expr + c_LOS * z, GRB.MINIMIZE)
            
            for k in range(num_trucks_chosen):
                depot_k = depot_nodes[k]
                m.addConstr(quicksum(x[depot_k, j, k] for j in nodes_dict[k] if j != depot_k) >= 1,
                            name=f"depot_departure_{k}")
                m.addConstr(quicksum(x[i, depot_k, k] for i in nodes_dict[k] if i != depot_k) >= 1,
                            name=f"depot_return_{k}")
            for k in range(num_trucks_chosen):
                for i in nodes_dict[k]:
                    if isinstance(i, int):
                        continue
                    m.addConstr(quicksum(x[i, j, k] for j in nodes_dict[k] if j != i) -
                                quicksum(x[j, i, k] for j in nodes_dict[k] if j != i) == 0,
                                name=f"flow_{i}_{k}")
            for t in tasks:
                m.addConstr(quicksum(x[t, j, k] for k in range(num_trucks_chosen) for j in nodes_dict[k] if j != t) == 1,
                            name=f"coverage_{t}")
            for k in range(num_trucks_chosen):
                for i in tasks:
                    for j in tasks:
                        if i == j:
                            continue
                        m.addConstr(u_vars[i] + plow_cost[i] + d_arc[(i, j, k)]
                                    <= u_vars[j] + M_val*(1 - x[i, j, k]),
                                    name=f"order_{i}_{j}_{k}")
            for k in range(num_trucks_chosen):
                depot_k = depot_nodes[k]
                for j in tasks:
                    m.addConstr(u_vars[j] >= d_arc[(depot_k, j, k)] - M_val*(1 - x[depot_k, j, k]),
                                name=f"depot_to_task_order_{j}_{k}")
            for k in range(num_trucks_chosen):
                m.addConstr(quicksum(d_arc[(i, j, k)] * x[i, j, k]
                                     for i in nodes_dict[k] for j in nodes_dict[k] if i != j) <= z,
                            name=f"longest_route_{k}")
            m.update()
            
            lp_filename = os.path.join(sample_folder, f"{variation_id}.lp")
            mps_filename = os.path.join(sample_folder, f"{variation_id}.mps")
            m.write(lp_filename)
            m.write(mps_filename)
            print(f"Variation {variation_id}: Model files written.")
            
            constrs = m.getConstrs()
            vars_model = m.getVars()
            A = lil_matrix((len(constrs), len(vars_model)))
            for i, constr in enumerate(constrs):
                for j, var in enumerate(vars_model):
                    coeff = m.getCoeff(constr, var)
                    if coeff != 0:
                        A[i, j] = coeff
            pickle_filename = os.path.join(sample_folder, f"{variation_id}_A.pkl")
            with open(pickle_filename, "wb") as f:
                pickle.dump(A, f)
            A_csr = A.tocsr()
            npz_filename = os.path.join(sample_folder, f"{variation_id}_A.npz")
            save_npz(npz_filename, A_csr)
            print(f"Variation {variation_id}: Coefficient matrix saved.")
            
            try:
                print(f"Variation {variation_id}: Solving model...")
                m.optimize()
            except GurobiError as e:
                if "size-limited license" in str(e).lower():
                    print(f"Variation {variation_id}: Model too large. Saving subgraph image with depot markers.")
                    fig_fail, ax_fail = ox.plot_graph(subgraph, figsize=(6,6), node_size=5, edge_linewidth=0.5, show=False, close=False)
                    for d in depot_nodes:
                        ax_fail.scatter(subgraph.nodes[d]['x'], subgraph.nodes[d]['y'], marker='*', c='yellow', s=300, edgecolors='black', zorder=5)
                    fail_img = os.path.join(sample_folder, f"{variation_id}_too_large.png")
                    fig_fail.savefig(fail_img)
                    plt.close(fig_fail)
                    continue
            if m.status == GRB.INFEASIBLE:
                print(f"Variation {variation_id}: Model infeasible. Saving subgraph image with depot markers.")
                fig_fail, ax_fail = ox.plot_graph(subgraph, figsize=(6,6), node_size=5, edge_linewidth=0.5, show=False, close=False)
                for d in depot_nodes:
                    ax_fail.scatter(subgraph.nodes[d]['x'], subgraph.nodes[d]['y'], marker='*', c='yellow', s=300, edgecolors='black', zorder=5)
                fail_img = os.path.join(sample_folder, f"{variation_id}_infeasible.png")
                fig_fail.savefig(fail_img)
                plt.close(fig_fail)
                continue
            
            optimal_obj = m.objVal if m.status == GRB.OPTIMAL else None
            print(f"Variation {variation_id}: Optimal objective = {optimal_obj}")
            
            # Build route_components using active arcs.
            route_components = {}
            for k in range(num_trucks_chosen):
                route_components[k] = []
                for i in nodes_dict[k]:
                    for j in nodes_dict[k]:
                        if i != j and (i, j, k) in x and x[i, j, k].X > 0.5:
                            route_components[k].append(i)
                            route_components[k].append(j)
            # Construct truck_paths from route_components.
            truck_paths = {}
            for k in range(num_trucks_chosen):
                depot_node = depot_nodes[k]
                temp = route_components[k].copy()
                num_depot_visits = temp.count(depot_node) / 2
                route = [depot_node]
                current = depot_node
                num_depot_visits -= 1
                while True:
                    next_node = None
                    for j in temp:
                        if current == j:
                            continue
                        if (current, j, k) in x and x[current, j, k].X > 0.5:
                            next_node = j
                            break
                    if next_node is None or (next_node == depot_node and num_depot_visits == 0):
                        break
                    if next_node == depot_node:
                        num_depot_visits -= 1
                    route.append(next_node)
                    current = next_node
                    temp.remove(next_node)
                if route[-1] != depot_node:
                    route.append(depot_node)
                truck_paths[k] = route
                print(f"Truck {k} connected route: {route}")
            
            # Compute per-truck distances.
            truck_total_deadhead = {}
            truck_total_distance = {}
            truck_deadhead_edges = {}
            for k in range(num_trucks_chosen):
                route = truck_paths[k]
                deadhead = 0
                deadhead_edges = []
                for i in range(len(route) - 1):
                    arc = (route[i], route[i+1], k)
                    dist_val = d_arc.get((route[i], route[i+1], k), 0)
                    deadhead += dist_val
                    deadhead_edges.append((route[i], route[i+1]))
                # Sum plowing cost for task edges (non-int).
                plowing = sum(plow_cost[node] for node in route if not isinstance(node, int))
                total = deadhead + plowing
                truck_total_deadhead[k] = deadhead
                truck_total_distance[k] = total
                truck_deadhead_edges[k] = deadhead_edges
                print(f"Truck {k}: Deadhead = {deadhead}, Plowing = {plowing}, Total = {total}")
            
            # Save animation.
            ani = animate_truck_routes(subgraph, truck_paths, {k: depot_nodes[k] for k in range(num_trucks_chosen)}, interval=1000)
            ani_filename = os.path.join(sample_folder, f"{variation_id}_animation.mp4")
            try:
                ani.save(ani_filename, writer="ffmpeg")
            except Exception as e:
                print(f"Could not save animation for {variation_id}: {e}")
            
            # Save images.
            fig_routes, ax_routes = plot_truck_routes_from_model(subgraph, truck_paths, {k: depot_nodes[k] for k in range(num_trucks_chosen)})
            combined_routes_filename = os.path.join(sample_folder, f"{variation_id}_truck_routes.png")
            fig_routes.savefig(combined_routes_filename)
            plt.close(fig_routes)
            for k in range(num_trucks_chosen):
                fig_single, ax_single = plot_single_truck_route_from_model(subgraph, k, truck_paths[k], depot_nodes[k])
                single_route_filename = os.path.join(sample_folder, f"{variation_id}_truck_{k}_route.png")
                fig_single.savefig(single_route_filename)
                plt.close(fig_single)
            fig_depots, ax_depots = ox.plot_graph(subgraph, figsize=(6,6), node_size=5, edge_linewidth=0.5, show=False, close=False)
            for k in range(num_trucks_chosen):
                d = depot_nodes[k]
                ax_depots.scatter(subgraph.nodes[d]['x'], subgraph.nodes[d]['y'], marker='*', c='yellow', s=300, edgecolors='black', zorder=5)
            depots_img_filename = os.path.join(sample_folder, f"{variation_id}_subgraph_depots.png")
            fig_depots.savefig(depots_img_filename)
            plt.close(fig_depots)
            
            # Build CSV row.
            row = [
                sample_id,
                num_nodes,
                num_edges,
                str(nodes_sub),
                str(edges_sub),
                str(edge_lengths),
                num_trucks_chosen,
                str(depot_nodes),
                str(tasks),
                str(task_nodes),
                str(truck_paths),
                str(truck_total_distance),
                max(truck_total_distance.values(), default=None),
                z.X if hasattr(z, "X") else None,
                str(truck_total_deadhead),
                str(truck_deadhead_edges),
                optimal_obj
            ]
            
            writer.writerow(row)
            csvfile.flush()
            valid_sample_count += 1
            print(f"  → Wrote {variation_id} to CSV.")
            
            valid_variation_found = True
            print(f"Variation {variation_id}: All outputs generated and saved.")
        
        if not valid_variation_found:
            print(f"Sample {sample_id}: No valid variations. Saving subgraph image (no_result).")
            fig_fail, ax_fail = ox.plot_graph(subgraph, figsize=(6,6), node_size=5, edge_linewidth=0.5, show=False, close=False)
            for d in depot_nodes:
                ax_fail.scatter(subgraph.nodes[d]['x'], subgraph.nodes[d]['y'], marker='*', c='yellow', s=300, edgecolors='black', zorder=5)
            fail_img = os.path.join(sample_folder, f"{sample_id}_no_result.png")
            fig_fail.savefig(fail_img)
            plt.close(fig_fail)
            
    csvfile.close()
    print(f"\nDone: {valid_sample_count} variations written to {csv_filename}")

In [1]:
ground_truth_generator_from_gurobi("Tartu, Estonia", radius=300, min_trucks=2, max_trucks=3,
                                    num_samples=2000, variations_per_sample=3, seed=10)